<a href="https://colab.research.google.com/github/arjunjyothieswarb/CS5180/blob/main/Assignment5/ex5_starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question 4 and 5 Windy Gridworld Domain

In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt

In [6]:
class WindyGridWorld(object):
    def __init__(self, enable_king_move=False, enable_no_move=False):
        # define the grid space
        self.grid = np.zeros((7, 10))

        # define the state space
        self.state_space = [[r, c] for r, c in zip(np.where(self.grid == 0.0)[0],
                                                   np.where(self.grid == 0.0)[1])]

        # define the start state
        self.start_state = [3, 0]

        # define the goal state
        self.goal_state = [3, 7]

        # define the wind
        self.wind = np.array([0, 0, 0, 1, 1, 1, 2, 2, 1, 0], dtype=int)

        # define the action space
        self.action_space = {
            "up": np.array([-1, 0]),
            "down": np.array([1, 0]),
            "left": np.array([0, -1]),
            "right": np.array([0, 1])
        }

        # Enable King's moves (Comment out the above action space to create a new one for King's moves)
        # if enable_king_move:

        #     if enable_no_move:
        #         #TODO
        #     else:
        #         #TODO
        #
        # else:
        #     #TODO


        # track the current state, time step, and action
        self.state = None
        self.t = None
        self.act = None


    def reset(self):
        # reset the agent to the start state
        self.state = self.start_state
        # reset the time step tracker
        self.t = 0
        # reset the action tracker
        self.act = None
        # reset the terminal flag
        terminated = False
        return self.state, terminated

    def step(self, act):

        #TODO
        terminated = False

        # Getting the action
        action = self.action_space[act]

        # Updating the state
        self.state[0] += action[0] + self.wind[self.state[1]]
        self.state[1] += action[1]

        # Clipping the state values
        self.state[0] = min(self.state[0], 6)
        self.state[0] = max(self.state[0], 0)

        self.state[1] = min(self.state[1], 9)
        self.state[1] = max(self.state[1], 0)

        # Checking if the agent has reached the goal state
        if self.state == self.goal_state:
            reward = 0
            terminated = True
        else:
            reward = -1

        self.t += 1
        self.act = act

        return self.state, reward, terminated

    def next_state(self, state, act):

        # Getting the action
        action = self.action_space[act]

        # Updating the state
        state[0] += action[0] + self.wind[self.state[1]]
        state[1] += action[1]

        # Clipping the state values
        state[0] = min(state[0], 6)
        state[0] = max(state[0], 0)

        state[1] = min(state[1], 9)
        state[1] = max(state[1], 0)

        return state

    def render(self):
        # plot the agent and the goal
        # agent = 1
        # goal = 2
        plot_arr = self.grid.copy()
        plot_arr[self.state[0], self.state[1]] = 1.0
        plot_arr[self.goal_state[0], self.goal_state[1]] = 2.0
        plt.clf()
        fig, arr = plt.subplots(1, 1)
        arr.set_title(f"state={self.state}, act={self.act}")
        arr.imshow(plot_arr)
        plt.show(block=False)
        plt.pause(1)
        plt.close(fig)


In [7]:
windy = WindyGridWorld()
print(windy.state_space)

[[0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [0, 6], [0, 7], [0, 8], [0, 9], [1, 0], [1, 1], [1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [1, 7], [1, 8], [1, 9], [2, 0], [2, 1], [2, 2], [2, 3], [2, 4], [2, 5], [2, 6], [2, 7], [2, 8], [2, 9], [3, 0], [3, 1], [3, 2], [3, 3], [3, 4], [3, 5], [3, 6], [3, 7], [3, 8], [3, 9], [4, 0], [4, 1], [4, 2], [4, 3], [4, 4], [4, 5], [4, 6], [4, 7], [4, 8], [4, 9], [5, 0], [5, 1], [5, 2], [5, 3], [5, 4], [5, 5], [5, 6], [5, 7], [5, 8], [5, 9], [6, 0], [6, 1], [6, 2], [6, 3], [6, 4], [6, 5], [6, 6], [6, 7], [6, 8], [6, 9]]


In [9]:
class SARSA(object):
    def __init__(self, env, alpha, epsilon, gamma, timeout):
        # define the parameters
        self.alpha = alpha
        self.epsilon = epsilon
        self.gamma = gamma

        # environment
        self.env = env

        # define the Q value table
        self.state_num = len(self.env.state_space)
        self.action_num = len(self.env.action_space.keys())
        self.Q = np.zeros((self.state_num, self.action_num))

        # define the timeout
        self.timeout = timeout

        # define action_space_index
        self.action_keys = list(self.env.action_space.keys())

        # define the action_key_index
        self.action_key_index = {"up":0, "down":1, "left":2, "right":3, "up-left":4, "up-right":5, "down-left":6, "down-right":7, "freeze":8}

    def behavior_policy(self, state):
        #TODO

        # Generating a random variable
        choice = np.random.random()

        # Exploration vs Exploitation
        if choice < self.epsilon:
            action = self.action_keys[np.random.randint(0,self.action_num)]
        else:
            Q_list = []
            for i in range(self.action_num):
                act = self.action_keys[i]

                state_prime = self.env.next_state(state, act)
                state_prime_index = (state_prime[0] * 10) + state_prime[1]

                Q_list.append(self.Q[state_prime][i])

            max_indices = np.argwhere(Q_list == np.max(Q_list)).T[0]
            action_index = np.random.choice(max_indices)

            action = self.action_keys[action_index]

        return action

    def update(self, s, a, r, s_prime, a_prime):
        #TODO

        # Getting the indices of states and actions
        state_index = (s[0] * 10) + s[1]
        state_prime_index = (s_prime[0] * 10) + s_prime[1]

        action_index = self.action_key_index[a]
        action_prime_index = self.action_key_index[a_prime]

        # Updating using SARSA
        self.Q[state_index][action_index] += self.alpha * (r + (self.gamma * self.Q[state_prime_index][action_prime_index]) - self.Q[state_index][action_index])

        return None

    def epsilon_greedy(self, state):

        # Generating a random variable
        choice = np.random.random()

        # Exploration vs Exploitation
        if choice < self.epsilon:
            action = self.action_keys[np.random.randint(0,self.action_num)]
        else:
            action = self.behavior_policy(state)

        return action

    def rollout(self):

        # Resetting the env
        state, terminated = self.env.reset()
        prev_state = state

        # Getting action
        action = self.epsilon_greedy(state)

        while(not terminated):

            # Checking for timeout
            if self.env.t > self.timeout:
                break

            # Taking the step
            state, reward, terminated = self.env.step(action)

            action_prime = self.epsilon_greedy(state)

            self.update(prev_state, action, reward, state, action_prime)

        return None

    def run(self):
        #TODO
        return None

In [ ]:
class ExpectedSARSA(object):
    def __init__(self, env, alpha, epsilon, gamma, timeout):
        # define the parameters
        self.alpha = alpha
        self.epsilon = epsilon
        self.gamma = gamma

        # environment
        self.env = env

        # define the Q value table
        self.state_num = len(self.env.state_space)
        self.action_num = len(self.env.action_space.keys())
        self.Q = np.zeros((self.state_num, self.action_num))

        # define the timeout
        self.timeout = timeout

    def behavior_policy(self, state):
        #TODO
        return None

    def update(self, s, a, r, s_prime, a_prime):
        #TODO
        return None

    def rollout(self):
        pass

    def run(self):
        #TODO
        return None

In [ ]:
class QLearning(object):
    def __init__(self, env, alpha, epsilon, gamma, timeout):
        # define the parameters
        self.alpha = alpha
        self.epsilon = epsilon
        self.gamma = gamma

        # environment
        self.env = env

        # define the Q value table
        self.state_num = len(self.env.state_space)
        self.action_num = len(self.env.action_space.keys())
        self.Q = np.zeros((self.state_num, self.action_num))

        # define the timeout
        self.timeout = timeout

    def behavior_policy(self, state):
        #TODO
        return None

    def update(self, s, a, r, s_prime):
        #TODO
        return None

    def rollout(self):
        pass

    def run(self):
        #TODO
        return None

In [ ]:
def plot_curves(arr_list, legend_list, color_list, ylabel):
    """
    Args:
        arr_list (list): list of results arrays to plot
        legend_list (list): list of legends corresponding to each result array
        color_list (list): list of color corresponding to each result array
        ylabel (string): label of the Y axis

    Make sure the elements in the arr_list, legend_list, and color_list are associated with each other correctly.
    Do not forget to change the ylabel for different plots.
    """
    # Clear the current figure
    plt.clf()
    fig, ax = plt.subplots(figsize=(12, 8))

    # Set labels
    ax.set_ylabel(ylabel)
    ax.set_xlabel("Time Steps")

    # Plot results
    h_list = []
    for arr, legend, color in zip(arr_list, legend_list, color_list):
        # Compute the mean and standard error while ignoring NaN values
        mean_arr = np.nanmean(arr, axis=0)
        arr_err = np.nanstd(arr, axis=0) / np.sqrt(np.sum(~np.isnan(arr), axis=0))

        # Plot the mean
        h, = ax.plot(range(len(mean_arr)), mean_arr, color=color, label=legend)

        # Plot the confidence band
        arr_err = 1.96 * arr_err  # 95% confidence interval
        ax.fill_between(range(len(mean_arr)),
                        mean_arr - arr_err,
                        mean_arr + arr_err,
                        alpha=0.3, color=color)
        # Save the plot handle
        h_list.append(h)

    # Set the title (adjust as needed)
    ax.set_title("Windy Gridworld Results")
    ax.legend(handles=h_list)
    plt.show()

In [ ]:
def run_on_policy_td_control(run_num, timeout):

    enable_king_move_actions = False
    enable_no_move_actions = False

    # create the environment
    env = WindyGridWorld(enable_king_move=enable_king_move_actions, enable_no_move=enable_no_move_actions)

    # parameters
    epsilon = 0.1
    alpha = 0.5
    gamma = 1.0

    # create the expected SARSA
    expected_sarsa_results_list = []
    for _ in range(run_num):
        # run for each trial
        controller_expected_sarsa = ExpectedSARSA(env, alpha, epsilon, gamma, timeout)
        episodes = controller_expected_sarsa.run()
        # append the results
        expected_sarsa_results_list.append(episodes[0:8000])

    # create the SARSA
    sarsa_results_list = []
    for _ in range(run_num):
        # run for each trial
        controller_sarsa = SARSA(env, alpha, epsilon, gamma, timeout)
        episodes = controller_sarsa.run()
        # append the results
        sarsa_results_list.append(episodes[0:8000])

    # create the Q learning
    q_learning_results_list = []
    for _ in range(run_num):
        # run for each trial
        controller_q_learning = QLearning(env, alpha, epsilon, gamma, timeout)
        episodes = controller_q_learning.run()
        # append the results
        q_learning_results_list.append(episodes[0:8000])

    sarsa_array = np.array(sarsa_results_list)
    expected_sarsa_array = np.array(expected_sarsa_results_list)
    q_learning_array = np.array(q_learning_results_list)

    # Plot the results
    plot_curves(
        [sarsa_array, expected_sarsa_array, q_learning_array],
        ['SARSA', 'Expected SARSA', 'Q-learning'],
        ['r', 'b', 'g'],
        "Episodes"
    )

In [ ]:
if __name__ == "__main__":
    # set randomness
    np.random.seed(1234)
    random.seed(1234)

    # trial number
    trial_num = 10
    # maximal time steps
    max_time_steps = 8000


    # run SARSA and Q Learning
    run_on_policy_td_control(trial_num, max_time_steps)

# Question 5 [5180 ONLY]
- Please refer to starter code from question 4 to help you get started. You will create your own TD(0) and Monte Carlo classes.
- We will continue with the original windy gridworld domain.
- A fixed policy π will be specified to collect episodes.
- A certain number of “training” episodes N ∈ {1, 10, 50} will be collected.
- Each method being investigated ( On-policy TD(0), On-policy Monte-Carlo prediction) will learn to      estimate the state-value.
function using the N “training“ episodes, respectively.
- We then evaluate the distribution of learning targets each method experiences at a specified state S. In
this question, S is the initial state marked as S in the Example 6.5.
- To do so, you need to collect additional 100 “evaluation” episodes. Instead of using these to perform
further updates to the state-value function, we will instead evaluate the distribution of learning targets
V(S) based on the “evaluation” episodes. For example, TD(0) will experience a set of {R+ V(S′)} targets,
whereas Monte-Carlo will experience a set of {G} targets.
- Note that in practice you should pre-collect both the training and evaluation episodes for efficiency and to
ensure consistency while comparing between different methods.

In [ ]:
def run_on_policy_mc_td_epsilon_greedy_windy_gridworld():
    enable_king_move_actions = False
    enable_no_move_actions = False

    # create environments
    env = WindyGridWorld(enable_king_move=enable_king_move_actions,
                         enable_no_move=enable_no_move_actions)
    env.reset()

In [ ]:
if __name__ == "__main__":
    # set randomness
    np.random.seed(1234)
    random.seed(1234)

    # run Monte Carlo and TD(0) for Question 6. Modify as necessary
    run_on_policy_mc_td_epsilon_greedy_windy_gridworld()